In [1]:
import random
import itertools
import functools
import time
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


import gudhi as gd
import torch
torch.set_default_dtype(torch.float64) 
import torch.nn as nn

In [2]:
# I WANT TO COMPARE CLOUDS WITH 1K POINTS WITH THE SAME STARTING POINT
def generate_orbits(n_points_per_orbit = 1000, params = [2.5, 3.5, 4.0, 4.1, 4.3], same_init_point = True):
    # create point clouds 
    ORBITS = np.zeros([len(params), n_points_per_orbit, 2])
    xcur_0, ycur_0 = np.random.rand(), np.random.rand() # not necesary to save the first one
    for id_pc, param in enumerate(params): # id_point_cloud
        if same_init_point:
            xcur, ycur = xcur_0, ycur_0 # not necesary to save the first one
        else:
            xcur, ycur =np.random.rand(), np.random.rand()
        for id_pt in range(n_points_per_orbit): # id_point
            xcur = (xcur + param * ycur * (1. - ycur)) % 1
            ycur = (ycur + param * xcur * (1. - xcur)) % 1
            ORBITS[id_pc, id_pt, :] = [xcur, ycur]
    return ORBITS

# function from [len(params), n_points, 2] to alpha-complex and persistence diagram
# to create PDs we need to: points -> skeleton(ac) -> simplex(st) -> persistence(pers)
# for each element of the dataset we'll have len(params) PDs to be compared
def extract_PD(cloud, id_class):
    """extract a dict 

    Args:
        cloud (_type_): array [1000,2] composing th ewhole point cloud
        id_class (_type_): index about the class of membership

    Returns:
        dict: with keys ['persist_0','persist_1','id_class']
    """
    # for every point cloud we create a dictionary storing the label and its persistence
    # usage of dictionary to store each other possible data linked to the point clous
    ac = gd.AlphaComplex(points=cloud)
    st = ac.create_simplex_tree()
    pers = st.persistence()
    
    #! TRANSPOSE TO HAVE THEN [BATCH SIZE, 2, NUM POINTS]
    pers_0 = np.array(st.persistence_intervals_in_dimension(0)).transpose()
    pers_1 = np.array(st.persistence_intervals_in_dimension(1)).transpose()
    pers_dict = {
        'persist_0': pers_0[:,:-1], # removing the last barcode, the one with inf
        'persist_1': pers_1, # here we should never have inf, since [0,1]^2 is compact/bounded  
        'persist': pers, # actual PD
        'id_class': id_class # label for classification
    }
    return pers_dict

def gaussian_transformation(pd):
    # Apply the function along the rows
    embs = np.apply_along_axis(gamma_p, axis=1, arr=pd)
    return torch.tensor(embs)

def gamma_p(p):
    # params of gaussian_transformation
    ts = torch.tensor([[0., 0.], \
          [0.25, 0.], [0.25, 0.25], \
          [0.5, 0.], [0.5, 0.25], [0.5, 0.5], \
          [0.75, 0.], [0.75, 0.25], [0.75, 0.5], [0.75, 0.75], \
          [1., 0.], [1., 0.25], [1., 0.5], [1., 0.75], [1., 1.]])
    sigma = 0.2
    # single point computaions for gaussian transformation
    squared_distances = torch.pow(ts - p, 2).sum(dim=1)
    emb = -squared_distances/(2*sigma**2)
    emb = torch.exp(emb)
    return emb



In [3]:
# hyper params
n_points = 1000
params = [2.5, 3.5, 4.0, 4.1, 4.3]
same_init_point = True
n_seq_per_dataset = [700, 300] # [size train, size test] for each class

extended_pers = False # to be implemented, at the moment not interested in

# init list fo persistence diagrams
pds_train = []
pds_test = []

# TRAIN 
for i in tqdm(range(n_seq_per_dataset[0]), desc='Create TRAIN Point Clouds'):
    ORBS = generate_orbits(n_points, params, same_init_point) # CREATE THE 5 POINT CLOUDS
    for j in range(ORBS.shape[0]):
        ij_pers = extract_PD(ORBS[j,:,:], j) # EXTRACT PDs
        pds_train.append(ij_pers) # STORE IN THE LIST pds_train

# TEST
for i in tqdm(range(n_seq_per_dataset[1]), desc='Create TEST Point Clouds'):
    ORBS = generate_orbits(n_points, params, same_init_point) # CREATE THE 5 POINT CLOUDS
    for j in range(ORBS.shape[0]):
        ij_pers = extract_PD(ORBS[j,:,:], j) # EXTRACT PDs
        pds_test.append(ij_pers) # STORE IN THE LIST pds_test

Create TRAIN Point Clouds:   0%|          | 1/700 [00:00<01:43,  6.77it/s]

Create TEST Point Clouds: 100%|██████████| 300/300 [00:45<00:00,  6.56it/s]


In [4]:
# type(pds_train[0])
# pds_train[0]['persist_0'] # H0 of a Pers Diagram
# len(pds_train[0]['persist_0']) # H0 of a Pers Diagram: [[births...], [deaths...]]

num_dgs_train = len(pds_train)
num_dgs_test = len(pds_test)

max_size_H0 = 999
max_size_train_H1 = max([len(pds_train[i]['persist_1'][0]) for i in range(num_dgs_train)])
max_size_test_H1 = max([len(pds_test[i]['persist_1'][0]) for i in range(num_dgs_test)])


In [35]:
# WHICH TRANSFORMATION DO WE WANT TO APPLY TO PDs?
# todo: HERE I WANT ALSO TO APPLY A DENSITY IN SOME CASES 

METHOD = 'Linear_length'
preproc_size = 750

if METHOD == 'Linear_length':
    
    train_vecs_H0 = np.zeros((num_dgs_train, max_size_H0))
    train_vecs_H1 = np.zeros((num_dgs_train, max_size_train_H1))
    test_vecs_H0 = np.zeros((num_dgs_test, max_size_H0))
    test_vecs_H1 = np.zeros((num_dgs_test, max_size_test_H1))
    
    # train
    barlength_train_H0 = [pds_train[i]['persist_0'][1] - pds_train[i]['persist_0'][0] for i in range(num_dgs_train)]
    barlength_train_H1 = [pds_train[i]['persist_1'][1] - pds_train[i]['persist_1'][0] for i in range(num_dgs_train)]
    # test
    barlength_test_H0 = [pds_test[i]['persist_0'][1] - pds_test[i]['persist_0'][0] for i in range(num_dgs_test)]
    barlength_test_H1 = [pds_test[i]['persist_1'][1] - pds_test[i]['persist_1'][0] for i in range(num_dgs_test)]

    # TRAIN
    for i in range(num_dgs_train):
        train_vecs_H0[i, :barlength_train_H0[i].shape[0]] = np.sort(barlength_train_H0[i])[::-1]
    for i in range(num_dgs_train):
        train_vecs_H1[i, :barlength_train_H1[i].shape[0]] = np.sort(barlength_train_H1[i])[::-1]
    # H0 TEST
    for i in range(num_dgs_test):
        test_vecs_H0[i, :barlength_test_H0[i].shape[0]] = np.sort(barlength_test_H0[i])[::-1]
    for i in range(num_dgs_test):
        test_vecs_H1[i, :barlength_test_H1[i].shape[0]] = np.sort(barlength_test_H1[i])[::-1]

    train_vecs_H0 = train_vecs_H0[:,:preproc_size]*1000 ##* ALTERNATIVE for TEMPERATURE??
    train_vecs_H1 = train_vecs_H1[:,:preproc_size]*100
    test_vecs_H0 = test_vecs_H0[:,:preproc_size]*1000
    test_vecs_H1 = test_vecs_H1[:,:preproc_size]*100

    # mean and std in training set
    train_mean = np.mean(train_vecs_H0, axis=0)
    train_std = np.std(train_vecs_H0, axis=0)


elif METHOD == 'Gauss_Transf':

    emb_dim = 15
    train_vecs_H0 = np.zeros((num_dgs_train, max_size_H0, emb_dim))
    train_vecs_H1 = np.zeros((num_dgs_train, max_size_train_H1, emb_dim))
    test_vecs_H0 = np.zeros((num_dgs_test, max_size_H0, emb_dim))
    test_vecs_H1 = np.zeros((num_dgs_test, max_size_test_H1, emb_dim))

    # about 15 minutes:
    # train
    barlength_train_H0 = [gaussian_transformation(np.array(pds_train[i]['persist_0']).transpose()) for i in range(num_dgs_train)] # shape (3500, 999, 15)
    barlength_train_H1 = [gaussian_transformation(np.array(pds_train[i]['persist_1']).transpose()) for i in range(num_dgs_train)]
    # test
    barlength_test_H0 = [gaussian_transformation(np.array(pds_test[i]['persist_0']).transpose()) for i in range(num_dgs_test)]
    barlength_test_H1 = [gaussian_transformation(np.array(pds_test[i]['persist_1']).transpose()) for i in range(num_dgs_test)]
    
    # # H0 TRAIN
    for i in range(num_dgs_train):
        train_vecs_H0[i, :barlength_train_H0[i].shape[0], :] = barlength_train_H0[i]
    # # H1 TRAIN
    for i in range(num_dgs_train):
        train_vecs_H1[i, :barlength_train_H1[i].shape[0],:] = barlength_train_H1[i]
    # # H0 TEST
    for i in range(num_dgs_test):
        test_vecs_H0[i, :barlength_test_H0[i].shape[0],:] = barlength_test_H0[i]
    for i in range(num_dgs_test):
        test_vecs_H1[i, :barlength_test_H1[i].shape[0],:] = barlength_test_H1[i]

    # here I can't create a matrix, here we have a tensor
    train_vecs_H0 = train_vecs_H0[:,:preproc_size,:]*1000 ##* ALTERNATIVE for TEMPERATURE??
    train_vecs_H1 = train_vecs_H1[:,:preproc_size,:]*100
    test_vecs_H0 = test_vecs_H0[:,:preproc_size,:]*1000
    test_vecs_H1 = test_vecs_H1[:,:preproc_size,:]*100


elif METHOD == 'Density':
    from scipy.stats import norm
    mu = 0.15
    sigma = 0.05

    train_vecs_H0 = np.zeros((num_dgs_train, max_size_H0))
    train_vecs_H1 = np.zeros((num_dgs_train, max_size_train_H1))
    test_vecs_H0 = np.zeros((num_dgs_test, max_size_H0))
    test_vecs_H1 = np.zeros((num_dgs_test, max_size_test_H1))

    barlength_train_H0 = [norm.cdf(np.array(pds_train[i]['persist_0'][1]), loc=mu, scale=sigma) - norm.cdf(np.array(pds_train[i]['persist_0'][0]), loc=mu, scale=sigma) for i in range(num_dgs_train)]
    barlength_train_H1 = [norm.cdf(np.array(pds_train[i]['persist_1'][1]), loc=mu, scale=sigma) - norm.cdf(np.array(pds_train[i]['persist_1'][0]), loc=mu, scale=sigma) for i in range(num_dgs_train)]
    barlength_test_H0 = [norm.cdf(np.array(pds_test[i]['persist_0'][1]), loc=mu, scale=sigma) - norm.cdf(np.array(pds_test[i]['persist_0'][0]), loc=mu, scale=sigma) for i in range(num_dgs_test)]
    barlength_test_H1 = [norm.cdf(np.array(pds_test[i]['persist_1'][1]), loc=mu, scale=sigma) - norm.cdf(np.array(pds_test[i]['persist_1'][0]), loc=mu, scale=sigma) for i in range(num_dgs_test)]
    
    # TRAIN
    for i in range(num_dgs_train):
        train_vecs_H0[i, :barlength_train_H0[i].shape[0]] = barlength_train_H0[i][::-1]
    for i in range(num_dgs_train):
        train_vecs_H1[i, :barlength_train_H1[i].shape[0]] = barlength_train_H1[i][::-1]
    # H0 TEST
    for i in range(num_dgs_test):
        test_vecs_H0[i, :barlength_test_H0[i].shape[0]] = barlength_test_H0[i][::-1]
    for i in range(num_dgs_test):
        test_vecs_H1[i, :barlength_test_H1[i].shape[0]] = barlength_test_H1[i][::-1]
    
    train_vecs_H0 = train_vecs_H0[:,:preproc_size]*1000 ##* ALTERNATIVE for TEMPERATURE??
    train_vecs_H1 = train_vecs_H1[:,:preproc_size]*100
    test_vecs_H0 = test_vecs_H0[:,:preproc_size]*1000
    test_vecs_H1 = test_vecs_H1[:,:preproc_size]*100

# mean and std in training set
# I have a matrix with gaussian transformation
train_mean = np.mean(train_vecs_H0, axis=0)
train_std = np.std(train_vecs_H0, axis=0)
        

In [6]:
train_mean = np.mean(train_mean, axis=0)
train_std = np.std(train_std, axis=0)

In [36]:
train_vecs_H0.shape, train_vecs_H1.shape, test_vecs_H0.shape, test_vecs_H1.shape
# ((3500, 750, 15), (3500, 750, 15), (1500, 750, 15), (1500, 750, 15)) if METHOD == 'Gauss_Transf'

((3500, 750), (3500, 750), (1500, 750), (1500, 750))

In [37]:
train_classes = np.array([pds_train[i]['id_class'] for i in range(num_dgs_train)])
test_classes = np.array([pds_test[i]['id_class'] for i in range(num_dgs_test)])
train_classes.shape, test_classes.shape

((3500,), (1500,))

In [38]:
# batching train and test datasets with respective labels
batch_size = 128

if METHOD == 'Gauss_Transf':
    n_train_batches = len(train_vecs_H0)//batch_size + 1
    batched_train = []
    for i in range(n_train_batches):
        batched_train.append(\
            [torch.tensor(train_vecs_H0[i*batch_size:(i+1)*batch_size,:,:]),\
            torch.tensor(train_vecs_H1[i*batch_size:(i+1)*batch_size,:,:]),\
            torch.tensor(train_classes[i*batch_size:(i+1)*batch_size])])
        
    n_test_batches = len(test_vecs_H0)//batch_size + 1
    batched_test = []
    for i in range(n_test_batches):
        batched_test.append(\
            [torch.tensor(test_vecs_H0[i*batch_size:(i+1)*batch_size,:,:]),\
            torch.tensor(test_vecs_H1[i*batch_size:(i+1)*batch_size,:,:]),\
            torch.tensor(test_classes[i*batch_size:(i+1)*batch_size])])

else:
    n_train_batches = len(train_vecs_H0)//batch_size + 1
    batched_train = []
    for i in range(n_train_batches):
        batched_train.append(\
            [torch.tensor(train_vecs_H0[i*batch_size:(i+1)*batch_size,:]),\
            torch.tensor(train_vecs_H1[i*batch_size:(i+1)*batch_size,:]),\
            torch.tensor(train_classes[i*batch_size:(i+1)*batch_size])])
        
    n_test_batches = len(test_vecs_H0)//batch_size + 1
    batched_test = []
    for i in range(n_test_batches):
        batched_test.append(\
            [torch.tensor(test_vecs_H0[i*batch_size:(i+1)*batch_size,:]),\
            torch.tensor(test_vecs_H1[i*batch_size:(i+1)*batch_size,:]),\
            torch.tensor(test_classes[i*batch_size:(i+1)*batch_size])])
n_train_batches, n_test_batches

(28, 12)

PERSISTENCE IMAGE $L_\infty$ Network ---

In [39]:
class linfLayer(nn.Module):
    def __init__(self, in_blocks, out_blocks, layer_norm_params=None):
        super().__init__()
        #todo: add mean and variance of the dataset

        if layer_norm_params is not None:
          mean = torch.tensor(layer_norm_params[0])
          std = torch.tensor(layer_norm_params[1])
          params = mean + torch.randn((out_blocks, in_blocks)) * std # normalization according to training values
          self.params = torch.nn.Parameter(data=params)
        else:
          params = torch.randn((out_blocks, in_blocks))
          self.params = torch.nn.Parameter(data=params)

    def forward(self, x, p):
        res = torch.cdist(x, self.params, p=p) #torch.cdist but with custom params
        return res

In [15]:
epochs = 100
inf=float("inf")

# p_schedule = np.ones(epochs)*inf
e1, e2 = 20,40 # e1+e2 < epochs !!
p_schedule = np.ones(epochs)*8
p_schedule[e1:e1+e2] = np.linspace(8, 100, e2)
p_schedule[e1+e2:] = inf

save_folder = f'./LinfModels/'

In [45]:
class Linf_PersImage(nn.Module):
    def __init__(self, emb_dim, hidden_size, topK, mean, std) -> None:
        super().__init__()
        self.mean = mean
        self.std = std
        self.topK = topK

        self.layer1_H0 = linfLayer(in_blocks=emb_dim, out_blocks=150, layer_norm_params=[train_mean, train_std])
        self.layer2_H0 = linfLayer(in_blocks=150, out_blocks=80)
        # self.layer3_H0 = linfLayer(in_blocks=30, out_blocks=20)
        self.layer4_H0 = linfLayer(in_blocks=80, out_blocks=hidden_size)
        
        self.layer1_H1 = linfLayer(in_blocks=emb_dim, out_blocks=150, layer_norm_params=[train_mean, train_std])
        self.layer2_H1 = linfLayer(in_blocks=150, out_blocks=80)
        # self.layer3_H1 = linfLayer(in_blocks=30, out_blocks=20)
        self.layer4_H1 = linfLayer(in_blocks=80, out_blocks=hidden_size)

        # self.layer1_concat = linfLayer(topK*(hidden_size*2), 80)
        self.layer1_concat = linfLayer(topK*2, 80)
        self.layer2_concat = linfLayer(80, 20)
        self.layer3_concat = linfLayer(20, 5)

    def forward(self, h0, h1, p):
        B = h0.shape[0]
        h0 = self.layer1_H0(h0, p)
        h0 = h0 - h0.mean(axis=0)
        h0 = self.layer2_H0(h0, p)
        h0 = h0 - h0.mean(axis=0)
        # h0 = self.layer3_H0(h0, p)
        # h0 = h0 - h0.mean(axis=0)
        h0 = self.layer4_H0(h0, p)
        h0 = torch.topk(h0, self.topK, dim=1)[0].view(B,-1)
        
        h1 = self.layer1_H1(h1, p)
        h1 = h1 - h1.mean(axis=0)
        h1 = self.layer2_H1(h1, p)
        h1 = h1 - h1.mean(axis=0)
        # h1 = self.layer3_H1(h1, p)
        # h1 = h1 - h1.mean(axis=0)
        h1 = self.layer4_H1(h1, p)
        h1 = torch.topk(h1, self.topK, dim=1)[0].view(B,-1)

        concat = torch.cat((h0,h1), dim = 1)
        # print(concat.shape)
        concat = self.layer1_concat(concat, p)
        concat = concat - concat.mean(axis=0)
        concat = self.layer2_concat(concat, p)
        concat = concat - concat.mean(axis=0)
        output = self.layer3_concat(concat, p)

        return -output

In [27]:
def test_model(model, test_data, if_plot):
    # Set the model to evaluation mode
    model.eval()

    target_labs = np.array([])
    pred_labs = np.array([])
    correct = 0
    total = 0

    # Disable gradient calculation
    with torch.no_grad():
        # for batch in tqdm(test_data):
        for (h0, h1, cl) in test_data:

            # Forward pass
            outputs = model(h0, h1, inf)
            # Get predicted labels
            _, predicted = torch.max(outputs.data, 1)
            # Total number of labels
            total += cl.size(0)
            # Total correct predictions
            correct += (predicted == cl).sum().item()
            target_labs = np.append(target_labs, cl)
            pred_labs = np.append(pred_labs, predicted)

    # Calculate accuracy
    accuracy = 100 * correct / total
    # print('Accuracy on the test set: {:.2f}%'.format(accuracy))

    if if_plot:
        cm = confusion_matrix(np.array(target_labs), np.array(pred_labs))
        classes = [1,2,3,4,5]
        # Plot confusion matrix
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()
        
    return accuracy

In [46]:
model = Linf_PersImage(preproc_size, 15, 10, train_mean, train_std)
# Model , Optimizer, Loss
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.05, eps=1e-06)
# list(model.parameters())[0].shape
losses = []
best_acc = 0.0

for i in range(epochs):
    for (h0, h1, cl) in batched_train:
        model.train()
        tot_loss = 0.0
        
        #calculate output
        outputs = model(torch.tensor(h0), torch.tensor(h1), p_schedule[i])

        #calculate loss
        loss = loss_fn(outputs, cl.long())

        #backprop
        optimizer.zero_grad()
        tot_loss += float(loss)
        loss.backward()
        optimizer.step()
    
    losses.append(tot_loss)
    test_acc = test_model(model, batched_test, if_plot=False)
    if best_acc < test_acc:
        if test_acc > 80.0:
            torch.save(model.state_dict(), save_folder + f'PURE_{preproc_size}_acc_{test_acc:.3f}.pth')
        print(f'\rEpoch {i+1:3}/{epochs} --> IMPROVEMENT from {best_acc:.3f} to {test_acc:.3f} \t')
        best_acc = test_acc
    else:
        # print(f'> Test Accuracy = {test_acc} [best = {best_acc}]')
        print(f"\r Epoch {i+1:3}\tloss : {loss:.5f}, p: {p_schedule[i]}", end='')

plt.plot(np.array(losses))
train_acc = test_model(model, batched_train, if_plot=True)
test_acc = test_model(model, batched_test, if_plot=True)
train_acc, test_acc

/tmp/ipykernel_418/2226993550.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = model(torch.tensor(h0), torch.tensor(h1), p_schedule[i])


Epoch   1/100 --> IMPROVEMENT from 0.000 to 52.333 	
Epoch   2/100 --> IMPROVEMENT from 52.333 to 57.267 	
Epoch   6/100 --> IMPROVEMENT from 57.267 to 59.000 	
Epoch  11/100 --> IMPROVEMENT from 59.000 to 59.133 	
 Epoch  23	loss : 1.23395, p: inf

KeyboardInterrupt: 